In [13]:
import pandas as pd
import json
import os


In [16]:
path = "D:\\phonepe_project\\pulse\\data\\map\\transaction\\hover\\country\\india\\state"
path_a = os.listdir(path)

In [17]:
std3 = {'state':[], 'year':[],'quater':[],'district':[], 'transaction_count':[], 'transaction_amount':[]}

for i in path_a:
    path_yr = path+"\\"+i+"\\"
    year = os.listdir(path_yr)

    for j in year:
        path_file = path_yr + j
        path_n = os.listdir(path_file)

        for k in path_n:
            asd = path_file + "\\" + k
            data_n = open(asd, "r")
            output = json.load(data_n)

            for l in output['data']['hoverDataList']:
                district = l['name']
                count = l['metric'][0]['count']
                amount = l['metric'][0]['amount']
                std3['transaction_type'].append(district)
                std3['transaction_count'].append(count)
                std3['transaction_amount'].append(amount)
                std3['state'].append(i)
                std3['year'].append(j)
                std3['quater'].append(int(k.strip('.json')))

df = pd.DataFrame(std3)


KeyError: 'transaction_type'

In [18]:
std3 = {'state': [], 'year': [], 'quater': [], 'district': [], 'transaction_count': [], 'transaction_amount': []}

for i in path_a:
    path_yr = path + "\\" + i + "\\"
    year = os.listdir(path_yr)

    for j in year:
        path_file = path_yr + j
        path_n = os.listdir(path_file)

        for k in path_n:
            asd = path_file + "\\" + k
            with open(asd, "r") as data_n:
                try:
                    output = json.load(data_n)
                except json.JSONDecodeError as e:
                    print(f"Error loading JSON in {asd}: {e}")
                    continue

            if output and output.get('data') and output['data'].get('hoverDataList'):
                for l in output['data']['hoverDataList']:
                    district = l.get('name', 'Unknown District')
                    count = l['metric'][0].get('count', 0)
                    amount = l['metric'][0].get('amount', 0)
                    std3['district'].append(district)
                    std3['transaction_count'].append(count)
                    std3['transaction_amount'].append(amount)
                    std3['state'].append(i)
                    std3['year'].append(j)
                    std3['quater'].append(int(k.strip('.json')))
            else:
                print(f"Missing or invalid data in {asd}")

df3 = pd.DataFrame(std3)


In [19]:
df3

,state,year,quater,district,transaction_count,transaction_amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
16827,west-bengal,2023,3,nadia district,30146530,5.787869e+10
16828,west-bengal,2023,3,birbhum district,17597047,3.090737e+10
16829,west-bengal,2023,3,purba medinipur district,34244893,6.813718e+10
16830,west-bengal,2023,3,maldah district,28539407,5.299766e+10


In [20]:
df3.to_csv(r'D:\phonepe_project\map_trans.csv')

In [21]:
import psycopg2

mydb = psycopg2.connect(
    host = "localhost",
    database = "postgres",
    user = "postgres",
    password = "123456")

mycursor = mydb.cursor()

In [23]:
query3 = """
            CREATE TABLE IF NOT EXISTS map_trans(
                state VARCHAR(255),
                year VARCHAR(4),
                quater INT, 
                district VARCHAR(255), 
                transaction_count INT, 
                transaction_amount NUMERIC
                )

            """ 

try:
    mycursor.execute(query3)
    mydb.commit()
    print("Tables created successfully.")
except Exception as e:
    print("Error:", e)
    mydb.rollback()

Tables created successfully.


In [24]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL database connection parameters
db_config = {
    'user': 'postgres',
    'password': '123456',
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres',
}

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:{db_config["port"]}/{db_config["database"]}')

# CSV file path
csv_file_path = 'D:\\phonepe_project\\map_trans.csv'

# Read CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Insert data into PostgreSQL table
try:
    df.to_sql('map_trans', con=engine, if_exists='replace', index=True)
    print("Data inserted successfully.")
except Exception as e:
    print("Error:", e)
finally:
    engine.dispose()  # Close the database connection


Data inserted successfully.
